**Initialization**
- I use these three lines of code on top of my each notebooks because it will help to prevent any problems while reloading the same project. And the third line of code helps to make visualization within the notebook

In [1]:
#@ INITIALIZATION: 
%reload_ext autoreload
%autoreload 2
%matplotlib inline

**Downloading Libraries and Dependencies**
- I have downloaded all the libraries and dependencies required for the project in one particular cell.

In [6]:
#@ IMPORTING MODULES: UNCOMMENT BELOW:
# !pip install transformers[sentencepiece]
# !pip install datasets
import torch
import pandas as pd
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from datasets import get_dataset_config_names
from datasets import load_dataset

#@ IGNORING WARNINGS: 
import warnings
warnings.filterwarnings("ignore")

**The Dataset**
- We will use **SubjQA** dataset which consists of more than 10,000 customer reviews in English about products and services.

In [5]:
#@ LOADING THE DATASET:
domains = get_dataset_config_names("subjqa")                # Getting domain names. 
domains                                                     # Inspection.

['books', 'electronics', 'grocery', 'movies', 'restaurants', 'tripadvisor']

In [8]:
#@ LOADING THE DATASET:
subjqa = load_dataset("subjqa", name="electronics")         # Initializing electronics qa. 
print(subjqa["train"]["answers"][1])                        # Inspecting answers.

Reusing dataset subjqa (/root/.cache/huggingface/datasets/subjqa/electronics/1.1.0/e5588f9298ff2d70686a00cc377e4bdccf4e32287459e3c6baf2dc5ab57fe7fd)


  0%|          | 0/3 [00:00<?, ?it/s]

{'text': ['Bass is weak as expected', 'Bass is weak as expected, even with EQ adjusted up'], 'answer_start': [1302, 1302], 'answer_subj_level': [1, 1], 'ans_subj_score': [0.5083333253860474, 0.5083333253860474], 'is_ans_subjective': [True, True]}


In [10]:
#@ INSPECTING THE DATASET:
dfs = {split: dset.to_pandas() for split,dset in subjqa.flatten().items()}  # Initialization.
for split, df in dfs.items():
    print(f"Number of questions in {split}: {df['id'].nunique()}")          # Inspecton.

Number of questions in train: 1295
Number of questions in test: 358
Number of questions in validation: 255


In [16]:
#@ PREPARING THE DATASET: 
qa_cols = ["title", "question", "answers.text", "answers.answer_start", 
           "context"]                                                       # Initialization.
sample_df = dfs["train"][qa_cols].sample(2, random_state=7)                 # Initializing sample dataframe. 
sample_df                                                                   # Inspecting dataframe.

,title,question,answers.text,answers.answer_start,context
791,B005DKZTMG,Does the keyboard lightweight?,[this keyboard is compact],[215],I really like this keyboard. I give it 4 star...
1159,B00AAIPT76,How is the battery?,[],[],I bought this after the first spare gopro batt...


In [17]:
#@ INSPECTING THE ANSWER:
start_index = sample_df["answers.answer_start"].iloc[0][0]                  # Initializing starting index.
end_index = start_index + len(sample_df["answers.text"].iloc[0][0])         # Initializing ending index.
sample_df["context"].iloc[0][start_index:end_index]                         # Inspecting answer.

'this keyboard is compact'